In [72]:
import pandas as pd
import ast

In [73]:
# Load the character metadata
movie_metadata_columns = [
    "Wikipedia movie ID", "Freebase movie ID", "Movie name", "Movie release date",
    "Movie box office revenue", "Movie runtime", "Movie languages", "Movie countries", 
    "Movie genres"
]

movie_metadata = pd.read_csv('./data/movie.metadata.tsv', sep='\t', header=None, names=movie_metadata_columns, encoding='utf-8', error_bad_lines=False)

# Display the first few rows of the character metadata
movie_metadata.head()

C:\Users\arist\AppData\Local\Temp\ipykernel_33576\2831659894.py:8: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  movie_metadata = pd.read_csv('./data/movie.metadata.tsv', sep='\t', header=None, names=movie_metadata_columns, encoding='utf-8', error_bad_lines=False)


,Wikipedia movie ID,Freebase movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


In [74]:
def parse_encoded_col(encoded_str):
    try:
        return ast.literal_eval(encoded_str)
    except (ValueError, SyntaxError):
        return {}

In [75]:
# Apply the parse_encoded_col function to relevant columns
movie_metadata['Movie languages'] = movie_metadata['Movie languages'].apply(parse_encoded_col)
movie_metadata['Movie countries'] = movie_metadata['Movie countries'].apply(parse_encoded_col)
movie_metadata['Movie genres'] = movie_metadata['Movie genres'].apply(parse_encoded_col)

# Extract values from the dictionaries (as lists)
movie_metadata['Languages'] = movie_metadata['Movie languages'].apply(lambda x: list(x.values()))
movie_metadata['Countries'] = movie_metadata['Movie countries'].apply(lambda x: list(x.values()))
movie_metadata['Genres'] = movie_metadata['Movie genres'].apply(lambda x: list(x.values()))

# Display the parsed columns
movie_metadata[['Movie name', 'Languages', 'Countries', 'Genres']].head()


,Movie name,Languages,Countries,Genres
0,Ghosts of Mars,[English Language],[United States of America],"[Thriller, Science Fiction, Horror, Adventure,..."
1,Getting Away with Murder: The JonBenét Ramsey ...,[English Language],[United States of America],"[Mystery, Biographical film, Drama, Crime Drama]"
2,Brun bitter,[Norwegian Language],[Norway],"[Crime Fiction, Drama]"
3,White Of The Eye,[English Language],[United Kingdom],"[Thriller, Erotic thriller, Psychological thri..."
4,A Woman in Flames,[German Language],[Germany],[Drama]


In [76]:
genre_mapping_dict = {
    "Softcore Porn": ["Adult"],
    "Prison film": ["Prison"],
    "Pinku eiga": ["Adult", "Japanese", "Asian"],
    "Inventions & Innovations": None, 
    "Bruceploitation": ["Martial Arts", "Exploitation", "Low Budget"],
    "Tollywood": ["Indian", "Asian"],
    "Instrumental Music": ["Music/Dance"],
    "Splatter film": ["Horror"],
    "Female buddy film": ["Adventure", "Comedy", "Women"],
    "Detective fiction": ["Crime", "Thriller"],                 # 10
    "Dystopia": ["Dystopia"],
    "Black comedy": ["Comedy"],
    "History": ["History"],
    "Silhouette animation": ["Animation"],
    "Combat Films": ["Action"],
    "Educational": ["Educational"],
    "Heaven-Can-Wait Fantasies": None,
    "Outlaw": ["Crime"],
    "Marriage Drama": ["Drama", "Women"],
    "Health & Fitness": ["Sport"],                              # 20
    "Stand-up comedy": ["Comedy"],
    "Sci-Fi Adventure": ["Science Fiction", "Adventure"],
    "Road-Horror": ["Horror"],
    "Anthology": ["Anthology"],
    "Biography": ["Biography"],
    "Doomsday film": ["Dystopia"],
    "Malayalam Cinema": ["Indian"],
    "Musical comedy": ["Musical", "Comedy"],
    "Chick flick": ["Romance", "Teen", "Women"],
    "Action/Adventure": ["Action", "Adventure"],                # 30
    "Horror Comedy": ["Horror", "Comedy"],
    "Spaghetti Western": ["Western", "European"],
    "Crime Drama": ["Crime", "Drama"],
    "Monster": ["Horror"],
    "Kafkaesque": ["Dystopian", "Psychological"],
    "Filipino": ["Filipino", "Asian"],
    "Movie serial": ["Action"],
    "War effort": ["Politics", "War"],
    "Extreme Sports": ["Sports"],
    "Sex comedy": ["Adult", "Comedy"],                        # 40
    "Documentary": ["Documentary"], 
    "B-Western": ["Western", "Low Budget"],
    "Existentialism": ["Philosophical"],
    "Demonic child": ["Horror"],
    "Buddy film": ["Adventure", "Comedy"],
    "Early Black Cinema": ["Black Cinema"],
    "Therimin music": ["Music/Dance"],
    "Coming-of-age film": ["Social"],
    "Sexploitation": ["Adult", "Exploitation", "Low Budget"],
    "Outlaw biker film": ["Crime"],                         # 50
    "Gross out": ["Horror"],
    "Drama": ["Drama"],
    "Neorealism": ["Art Film"],
    "Christmas movie": ["Holidays"],
    "British New Wave": ["Art Film", "European"],
    "Comedy of Errors": ["Comedy"],
    "Computer Animation": ["Animation"],
    "Children's/Family": ["Kids", "Family"],
    "Surrealism": ["Art Film"],
    "Suspense": ["Thriller"],                               # 60
    "Dogme 95": ["Avant-Garde"],
    "Family Film": ["Family"],
    "Stop motion": ["Stop Motion"],
    "Mockumentary": ["Documentary"],
    "Ealing Comedies": ["Comedy"],
    "Czechoslovak New Wave": ["Art Film", "European"],
    "Archives and records": ["History"],
    "Social issues": ["Social"],
    "Environmental Science": ["Nature", "Educational"],
    "Short Film": ["Short Film"],                           # 70
    "Religious Film": ["Religious"],
    "The Netherlands in World War II": ["History", "War"],
    "Propaganda film": ["Politics"],
    "Historical Epic": ["History", "Epic"],
    "Action": ["Action"],
    "Horror": ["Horror"],
    "C-Movie": ["Low Budget"],
    "Film-Opera": ["Music/Dance"],
    "Period piece": ["History", "Drama"],
    "Silent film": ["Silent"],                              # 80
    "Political thriller": ["Politics", "Thriller"],
    "Absurdism": ["Art Film", "Philosophical"],
    "Gulf War": ["History", "War"],
    "Humour": ["Comedy"],
    "Sports": ["Sports"],
    "Bollywood": ["Indian"],
    "Historical Documentaries": ["History", "Documentary"],
    "Tokusatsu": ["Japanese"],
    "Road movie": ["Adventure"],
    "Conspiracy fiction": ["Politics", "Fiction"],          # 90
    "Punk rock": ["Music/Dance"],
    "Singing cowboy": ["Music/Dance", "Western"],
    "Breakdance": ["Music/Dance"],
    "Fictional film": ["Fiction"],
    "Feature film": None,
    "Epic": ["Epic"],
    "Journalism": ["Politics"],
    "Buddy Picture": ["Adventure", "Comedy"],
    "Children's Issues": ["Social"],
    "Family-Oriented Adventure": ["Family", "Adventure"],   # 100
    "Psychological thriller": ["Psychological", "Thriller"],
    "Children's": ["Kids"],
    "Z movie": ["Low Budget"],
    "Statutory rape": ["Adult"],
    "Mumblecore": ["Low Budget"],
    "Nature": ["Nature"],
    "Teen": ["Teen"],
    "Superhero movie": ["Action", "Science Fiction"],
    "Biographical film": ["Biography"],
    "British Empire Film": ["History", "War"],              # 110
    "Experimental film": ["Avant-Garde"],
    "Courtroom Drama": ["Drama"],
    "Comedy of manners": ["Comedy"],
    "Homoeroticism": ["LGBTQ", "Adult"],
    "Musical Drama": ["Musical", "Drama"],
    "Backstage Musical": ["Music/Dance"],
    "Screwball comedy": ["Romance", "Comedy"],
    "Libraries and librarians": None,
    "Erotica": ["Adult"],
    "Plague": ["Dystopian"],                                # 120
    "Martial Arts Film": ["Martial Arts"],
    "Slice of life story": ["Biography"],
    "Satire": ["Comedy"],
    "Animal Picture": ["Animation"],
    "LGBT": ["LGBTQ"],
    "Film adaptation": ["Film Adaptation"],
    "Alien Film": ["Horror"],
    "Boxing": ["Sports"],
    "Vampire movies": ["Horror"],
    "Parkour in popular culture": ["Sports"],               # 130
    "Whodunit": ["Crime"],
    "Revisionist Western": ["Western"],
    "World History": ["History"],
    "News": ["Politics"],
    "Political satire": ["Politics", "Comedy"],
    "Costume Horror": ["Horror"],
    "Linguistics": None,
    "Alien invasion": ["Horror"],
    "Indie": ["Independent"],
    "Master Criminal Films": ["Crime"],                     # 140
    "Anime": ["Animated", "Japanese"],
    "Haunted House Film": ["Horror"],
    "Baseball": ["Sport"],
    "Erotic thriller": ["Adult", "Thriller"],
    "B-movie": ["Low Budget"],
    "Foreign legion": ["War", "History"],
    "Biker Film": ["Sport"],
    "Political Documetary": ["Politics", "Documentary"],
    "Social problem film": ["Social"],
    "Detective": ["Crime"],                                 # 150
    "Blaxploitation": ["Black Cinema", "Exploitation", "Low Budget"],
    "Gangster Film": ["Crime"],
    "Education": ["Educational"],
    "Romantic drama": ["Romance", "Drama", "Women"],
    "Auto racing": ["Cars"],
    "Gender Issues": ["Gender Issues", "Social"],
    "Period Horror": ["Horror"],
    "Inspirational Drama": ["Drama"],
    "Fantasy": ["Fantasy"],
    "Airplanes and airports": None,                         # 160
    "Swashbuckler films": ["Action"],
    "Time travel": ["Science Fiction"],
    "Romantic fantasy": ["Romance", "Fantasy"],
    "Beach Party film": ["Beach"],
    "Comedy-drama": ["Comedy", "Drama"],
    "Americana": ["USA"],
    "Business": None,
    "Illnesses & Disabilities": ["Social", "Medical"],
    "Nuclear warfare": ["Dystopian", "War"],
    "Crime Thriller": ["Crime", "Thriller"],                # 170
    "Steampunk": ["Science Fiction", "Futuristic"],
    "Children's Entertainment": ["Kids"],
    "Anti-war": ["Social", "War"],
    "Star vehicle": ["Start Vehicle"],
    "Feminist Film": ["Feminism"],
    "Gay Themed": ["LGBTQ"],
    "World cinema": ["World Cinema"],
    "Chase Movie": ["Action", "Cars"],
    "Hip hop movies": ["Music/Dance", "Black Cinema"],
    "Gothic Film": ["Horror", "Art Film"],                  # 180
    "Children's Fantasy": ["Fantasy", "Kids"],
    "Film noir": ["Crime", "Drama"],
    "Romantic comedy": ["Romance", "Comedy"],
    "Western": ["Western"],
    "Caper story": ["Crime"],
    "Samurai cinema": ["Action", "Japanese"],
    "Animated Musical": ["Animated", "Music/Dance"],
    "Cult": ["Religious"],
    "Science Fiction": ["Science Fiction"],
    "Crime Fiction": ["Crime", "Fiction"],
    "Disaster": ["Dystopian"],                            # 190
    "Indian Western": ["Indian"],
    "Black-and-white": ["Black and White"],
    "Sponsored film": ["Sponsored"],
    "Psychological horror": ["Psychological", "Horror"],
    "Television movie": ["Television"],
    "Legal drama": ["Social", "Drama"],
    "Addiction Drama": ["Drama"],
    "Fan film": None,
    "Culture & Society": ["Social"],                  # 200
    "Sci Fi Pictures original films": ["Science Fiction"],
    "Heist": ["Crime"],
    "Docudrama": ["Documentary", "Drama"],
    "Buddy cop": ["Crime", "Adventure"],
    "Spy": ["Crime"],
    "Animation": ["Animation"],
    "Horse racing": ["Sports"],
    "Superhero": ["Action", "Science Fiction"],
    "Cold War": ["History"],
    "Dance": ["Music/Dance"],                      # 210
    "Goat gland": None,
    "Tragedy": ["Drama"],
    "Finance & Investing": None,
    "Film": None,
    "Anti-war film": ["Social", "War"],
    "Graphic & Applied Arts": None,
    "Pornography": ["Adult"],
    "Escape Film": ["Prison"],
    "Language & Literature": ["Literary"],
    "Bengali Cinema": ["Social", "Indian", "Asian"],                 # 220
    "Essay Film": ["Literary"],
    "Family & Personal Relationships": ["Social"],
    "Private military company": None,
    "Crime Comedy": ["Crime", "Comedy"],
    "Mystery": ["Thriller"],
    "Zombie Film": ["Horror", "Dystopian"],
    "Concert film": ["Music/Dance"],
    "Media Studies": ["Social"],
    "Anthropology": ["Social"],
    "Hybrid Western": ["Western"],
    "Heavenly Comedy": ["Comedy"],
    "Ensemble Film": ["Ensemble"],
    "Comedy": ["Comedy"],
    "Women in prison films": ["Prison"],
    "Space western": ["Western", "Science Fiction"],
    "Thriller": ["Thriller"],
    "Workplace Comedy": ["Comedy"],
    "Gay": ["LGBTQ"],
    "Animated cartoon": ["Animated"],
    "Fantasy Adventure": ["Fantasy", "Adventure"],
    "Music": ["Music/Dance"],
    "Supermarionation": None,
    "Adventure Comedy": ["Adventure", "Comedy"],
    "Art film": ["Art Film"],
    "Science fiction Western": ["Western", "Science Fiction"],
    "Sword and sorcery films": ["Science Fiction"],
    "Historical fiction": ["History", "Fiction"],
    "Glamorized Spy Film": ["Crime"],
    "Future noir": ["Crime", "Drama", "Science Fiction"],
    "Christian film": ["Religious"],
    "Childhood Drama": ["Drama"],
    "Erotic Drama": ["Adult", "Drama"],
    "Kitchen sink realism": ["Art Film"],
    "Fantasy Comedy": ["Fantasy", "Comedy"],
    "Clay animation": ["Stop Motion"],
    "Romantic thriller": ["Romance", "Thriller"],
    "Gay pornography": ["Adult", "LGBTQ"],
    "Filipino Movies": ["Filipino", "Asian"],
    "Comedy Thriller": ["Comedy", "Thriller"],
    "Biopic [feature]": ["Biography"],
    "Ninja movie": ["Action"],
    "Comedy Western": ["Comedy", "Western"],
    "Epic Western": ["Epic", "Western"],
    "Wuxia": ["Martial Arts", "Chinese"],
    "Prison escape": ["Prison"],
    "Tamil cinema": ["Indian"],
    "Pornographic movie": ["Adult"],
    "Media Satire": ["Politics", "Comedy"],
    "Supernatural": ["Science Fiction"],
    "Albino bias": None,
    "Movies About Gladiators": ["Action", "History"],
    "Action Comedy": ["Action", "Comedy"],
    "Werewolf fiction": ["Horror", "Fiction"],
    "Space opera": ["Science Fiction"],
    "Latino": ["Latino"],
    "Reboot": ["Reboot"],
    "Revisionist Fairy Tale": None,
    "Animals": ["Nature"],
    "Albino bias": None,
    "Action Comedy": ["Action", "Comedy"],
    "Werewolf fiction": ["Horror", "Fiction"],
    "Space opera": ["Science Fiction"],
    "Reboot": ["Reboot"],
    "Revisionist Fairy Tale": None,
    "Animals": ["Nature"],
    "Sci-Fi Thriller": ["Science Fiction", "Thriller"],
    "Political drama": ["Politics", "Drama"],
    "Mondo film": ["Documentary", "Horror"],
    "Costume Adventure": ["Adventure"],
    "Giallo": ["Thriller", "Psychological", "Horror"],
    "Sword and Sandal": ["History"],
    "Adult": ["Adult"],
    "Expressionism": ["Art Film"],
    "Gross-out film": ["Horror"],
    "Fairy tale": ["Science Fiction"],
    "Tragicomedy": ["Drama", "Comedy"],
    "Point of view shot": None,
    "Chinese Movies": ["Chinese"],
    "Slapstick": ["Comedy"],
    "Northern": ["Western"],
    "Sword and sorcery": ["Science Fiction", "Adventure"],
    "Prison": ["Prison"],
    "Courtroom Comedy": ["Social", "Comedy"],
    "Coming of age": ["Social"],
    "School story": ["Social"],
    "New Queer Cinema": ["LGBTQ"],
    "Comdedy": ["Comedy"],
    "Exploitation": ["Exploitation", "Low Budget"],
    "Crime": ["Crime"],
    "Revenge": ["Drama"],
    "Costume drama": ["Drama"],
    "Medical fiction": ["Medical", "Fiction"],
    "Avant-garde": ["Avant-Garde"],
    "Parody": ["Comedy"],
    "Action Thrillers": ["Action", "Thrillers"],
    "Apocalyptic and post-apocalyptic fiction": ["Dystopian", "Fiction"],
    "Hagiography": ["Religious"],
    "Creature Film": ["Horror"],
    "New Hollywood": ["USA"],
    "Adventure": ["Adventure"],
    "Jukebox musical": ["Music/Dance"],
    "Interpersonal Relationships": ["Social"],
    "Acid western": ["Western"],
    "Law & Crime": ["Social", "Crime"],
    "Juvenile Delinquency Film": ["Crime"],
    "Race movie": ["Cars"],
    "Natural disaster": ["Nature"],
    "Slasher": ["Horror"],
    "Live action": ["Animation"],
    "Monster movie": ["Horror"],
    "Jungle Film": ["Nature"],
    "Bloopers & Candid Camera": None,
    "Sci-Fi Horror": ["Science Fiction", "Horror"],
    "Operetta": ["Music/Dance", "Art Film"],
    "Fantasy Drama": ["Fantasy", "Drama"],
    "Melodrama": ["Drama"],
    "Stoner film": ["Comedy"],
    "Psycho-biddy": ["Horror", "Thriller"],
    "Rockumentary": ["Music/Dance", "Documentary"],
    "War film": ["War"],
    "Archaeology": None,
    "Holiday Film": ["Holidays"],
    "Filmed Play": None,
    "Family Drama": ["Drama"],
    "Natural horror films": ["Nature", "Horror"],
    "Musical": ["Music/Dance"],
    "Political cinema": ["Politics"],
    "Cyberpunk": ["Science Fiction", "Dystopian", "Futuristic"],
    "Romance Film": ["Romance"],
    "Comedy horror": ["Comedy", "Horror"],
    "Roadshow/Carny": ["Art Film"],
    "Remake": ["Remake"],
    "Domestic Comedy": ["Comedy"],
    "Beach Film": ["Beach"],
    "Pre-Code": ["USA"],
    "Roadshow theatrical release": None,
    "Comedy film": ["Comedy"],
    "Travel": None,
    "Computers": None,
    "Historical drama": ["History", "Drama"],
    "Film & Television History": ["History"],
    "Neo-noir": ["Crime", "Drama"],
    "Gay Interest": ["LBGTQ"],
    "Hardcore pornography": ["Adult"],
    "Mythological Fantasy": ["Fantasy", "Fiction"],
    "Camp": ["Art Film"],
    "Film à clef": ["Fiction"],
    "Patriotic film": ["USA", "Propaganda"],
    "Japanese Movies": ["Japanese", "Asian"],
    "Cavalry Film": ["Western", "USA"],
}

In [77]:
# Assuming movie_metadata is your existing DataFrame and it has a 'Genres' column with the old genres
# Also assuming genre_mapping_dict is the dictionary with your genre mappings

# Define the mapping function
def map_genres(old_genres_list, mapping_dict):
    new_genres_list = []
    for genre in old_genres_list:
        # Get the new genres from the dictionary, if not found or None, it will return an empty list
        mapped = mapping_dict.get(genre, [])
        if mapped is not None:
            new_genres_list.extend(mapped)
    # Return the unique genres after mapping
    return list(set(new_genres_list))

# Map the genres for each movie in the DataFrame
movie_metadata['Genres Reduced'] = movie_metadata['Genres'].apply(lambda x: map_genres(x, genre_mapping_dict))

# This will add the 'Genres Reduced' column to the movie_metadata DataFrame


In [78]:
movie_metadata.head()

,Wikipedia movie ID,Freebase movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres,Languages,Countries,Genres,Genres Reduced
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,{'/m/02h40lc': 'English Language'},{'/m/09c7w0': 'United States of America'},"{'/m/01jfsb': 'Thriller', '/m/06n90': 'Science...",[English Language],[United States of America],"[Thriller, Science Fiction, Horror, Adventure,...","[Thriller, Action, Adventure, Horror, Western,..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,{'/m/02h40lc': 'English Language'},{'/m/09c7w0': 'United States of America'},"{'/m/02n4kr': 'Mystery', '/m/03bxz7': 'Biograp...",[English Language],[United States of America],"[Mystery, Biographical film, Drama, Crime Drama]","[Thriller, Crime, Drama, Biography]"
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,{'/m/05f_3': 'Norwegian Language'},{'/m/05b4w': 'Norway'},"{'/m/0lsxr': 'Crime Fiction', '/m/07s9rl0': 'D...",[Norwegian Language],[Norway],"[Crime Fiction, Drama]","[Crime, Fiction, Drama]"
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,{'/m/02h40lc': 'English Language'},{'/m/07ssc': 'United Kingdom'},"{'/m/01jfsb': 'Thriller', '/m/0glj9q': 'Erotic...",[English Language],[United Kingdom],"[Thriller, Erotic thriller, Psychological thri...","[Thriller, Psychological, Adult]"
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,{'/m/04306rv': 'German Language'},{'/m/0345h': 'Germany'},{'/m/07s9rl0': 'Drama'},[German Language],[Germany],[Drama],[Drama]


In [79]:
# Export as csv file, only columns we need
movie_metadata[['Freebase movie ID', 'Movie name', 'Movie release date', 'Movie box office revenue', 'Movie runtime', 'Languages', 'Countries', 'Genres Reduced']].to_csv('./data/movie.metadata.v1.csv', index=False)


In [80]:
unique_languages = set(x for l in movie_metadata['Languages'] for x in l)
print('There are {} unique languages.'.format(len(unique_languages)))

# unique_languages[0]
print(unique_languages)
# for language in unique_languages:
#     print(language)

There are 207 unique languages.
{'Finnish Language', 'Quechua', 'Thai Language', 'Inuktitut', 'Indonesian Language', 'Swedish Language', 'French Sign Language', 'Catalan language', 'Hokkien', 'Hausa Language', 'Crow Language', 'Slovak Language', 'Japanese Language', 'Telugu language', 'Shanxi', 'Mohawk Language', 'Norwegian Language', 'Krio Language', 'Kinyarwanda language', 'Hindustani language', 'Shanghainese', 'Tamang language', 'Assamese Language', 'Tulu Language', 'Papiamento language', 'Hazaragi Language', 'Frisian, Western Language', 'Maninka language', 'German Language', 'Cantonese', 'Saami, North Language', 'Bengali Language', 'Danish Language', 'Mandarin Chinese', 'Bambara language', 'Old English language', 'Chechen Language', 'Yolngu Matha', 'Min Nan', 'Chinese, Jinyu Language', 'Dari', 'Hainanese', 'Serbo-Croatian', 'Thai, Northeastern Language', 'Australian Aboriginal Pidgin English', 'Kurdish language', 'Polish Language', 'Khmer, Central Language', 'Tamil Language', '\ud8

In [81]:
unique_genres = set(x for l in movie_metadata['Genres Reduced'] for x in l)
print('There are {} unique genres.'.format(len(unique_genres)))

print('All the genres:')
for genre in unique_genres:
    print(genre)

There are 79 unique genres.
All the genres:
Exploitation
Psychological
Film Adaptation
Holidays
Thriller
World Cinema
Television
Martial Arts
Comedy
Sport
Action
Epic
Literary
Horror
Japanese
Family
Biography
Art Film
Dystopian
Adventure
Fiction
USA
Sponsored
Western
Science Fiction
Independent
Dystopia
Romance
LBGTQ
Silent
Fantasy
Asian
Indian
Sports
Drama
War
Low Budget
Nature
Philosophical
Thrillers
Prison
Politics
Black and White
Beach
Avant-Garde
Reboot
Ensemble
Woman
Stop Motion
Crime
Start Vehicle
Latino
Documentary
Gender Issues
Medical
Cars
Chinese
Futuristic
Feminism
European
Remake
Teen
LGBTQ
Educational
Music/Dance
Short Film
Women
Propaganda
Black Cinema
Musical
History
Adult
Anthology
Kids
Animation
Filipino
Social
Animated
Religious


In [82]:
# Find Movies with zero genres
zero_genres_reduced = movie_metadata[movie_metadata['Genres Reduced'].map(len) == 0]
print('There are {} movies with zero genres.'.format(len(zero_genres_reduced)))

zero_genres = movie_metadata[movie_metadata['Genres'].map(len) == 0]
print('There were {} movies with zero genres.'.format(len(zero_genres)))

There are 2306 movies with zero genres.
There were 2294 movies with zero genres.
